In [ ]:
%cd ~
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell

topdir='/Users/cslage/Research/LSST/code/GUI/'
thedir=topdir+'profiles/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'

%matplotlib inline

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
1.0/Area(-0.5,0.5,-0.5,0.5,1.0,1.0,1.0)

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
zfilelist=sort(glob.glob(thedir+'114-04_spot-30um_light_3??_20150709??????.fits')+
               glob.glob(thedir+'114-04_spot-30um_light_4??_201507????????.fits'))
zfilelist

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    for i in [7]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-23:-20])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 10
    xcoomax = 490
    ycoomin = 100
    ycoomax = 1900
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename,segmentnumber)
    extname = hdr['EXTNAME']
    img=pyfits.getdata(fitsfilename,extname) 
    overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
    catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            spotlist.xoffset[n] = xcoord - xint - 1.0
            spotlist.yoffset[n] = ycoord - yint - 1.0     
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = overscansubimg[ymin:ymax, xmin:xmax]
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

import forward

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
nx = 9
ny = 9
numspots = 10
segmentnumber = 7

fitsfilename = zfilelist[0]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.9, 1.4)

print "nstamps = %d"%spotlist.nstamps

param0 = [1.00, 1.00]

starttime = time.time()
fom = FOM(param0)
elapsed = time.time() - starttime

print "CFom = %.1f, time = %.3f seconds"%(fom, elapsed)
for i in range(numspots):
    print "Imax = %.2f"%spotlist.imax[i]


In [ ]:
zfilelist[50]

In [ ]:
nx = 9
ny = 9
numspots = 1800
results = []
spotlists=[]
segmentnumber = 7

numtrials = 10

ssigxs = []

ssigys = []

for fitsfilename in zfilelist[40:50]:
    print fitsfilename
    fullspotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny,0.7,1.4)
    print "nstamps = %d"%fullspotlist.nstamps
    
    sigmaxs = []
    sigmays = []
  
    for trial in range(numtrials):
        stampxmin = -(int(nx/2)+0.5)
        stampxmax = -stampxmin
        stampymin = -(int(ny/2)+0.5)
        stampymax = -stampymin

        spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots/numtrials)
        for n in range(spotlist.nstamps):
            nfull = int((numspots-1) * rand())
            spotlist.xoffset[n] = fullspotlist.xoffset[nfull]
            spotlist.xoffset[n] = fullspotlist.xoffset[nfull]
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = fullspotlist.data[i,j,nfull]
 
        args=()
        param0 = [1.0, 1.0]
        Result = fmin_powell(FOM, param0, args, full_output=False)
        #print "Trial # %d"%trial, Result 
        sigmaxs.append(Result[0])
        sigmays.append(Result[1])

        ssigx = array(sigmaxs).std()/sqrt(numtrials)
        ssigy = array(sigmays).std()/sqrt(numtrials)
        
        ssigxs.append(ssigx)
        ssigys.append(ssigy)
        
    print "Sx sigma = %.4f, Sy sigma = %.4f"%(ssigx, ssigy)

In [ ]:
print array(ssigxs).mean()

print array(ssigys).mean()

In [ ]:
from scipy import stats
figure()
imaxs = []
sigmaxs = []
sigmays = []
for i,result in enumerate(results[0:93]):
    imax = spotlists[i].imax.mean()
    ADU_correction = Area(-0.5,0.5,-0.5,0.5,result[0],result[1],1.0)
    imaxs.append(imax * ADU_correction)
    sigmaxs.append(result[0])
    sigmays.append(result[1])
    #if result[2] < 1.06:
    #    print "I = %d, Imax = %.1f, sigmax = %.3f, sigmay = %.3f"%(i,result[0],result[1],result[2])
scatter(imaxs, sigmaxs, color = 'blue', lw = 2, label = 'Sigma-x')
scatter(imaxs, sigmays, color = 'green', lw = 2, label = 'Sigma-y')

slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs[15:50],sigmaxs[15:50])

xplot=linspace(-5000.0,60000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='red', lw = 2, ls = '--')
tslope = slope/intercept * 100.0 * 20000.0
text(0.0,1.1,"Slope = %.1f percent per 20,000 ADU"%tslope)

xlabel('Central Peak(ADUs)')
ylabel('Sigma (Pixels)')
legend(loc= 'lower right')
savefig("Forward_Model_Spots_Powell.png")

In [ ]:
figure()
testn = 45
spot = 1787
spotlist = spotlists[testn]
Result = results[testn]
Imax = spotlist.imax[spot]
sigmax = Result[0]
sigmay = Result[1]
seqno = int(zfilelist[testn].split("_")[3])
suptitle=("Sequence # %d, Spot # %d"%(seqno,spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)

subplots_adjust(wspace = 1.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot],interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot], lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
legend(loc = (-4.0,0.8))
savefig("Typical_Fit_Powell")


In [ ]:
print len(zfilelist)
print len(results)

In [ ]:
from scipy import stats
figure()
exp_current=[]
intensities=[]
for i,fitsfilename in enumerate(zfilelist[0:93]):
    try:
        exptime=pyfits.getheader(fitsfilename,0)['EXPTIME'] 
        mondiode=pyfits.getheader(fitsfilename,0)['MONDIODE'] 
        exp_current.append(float(exptime) * mondiode / (1.0E-9))
        ADU_correction = Area(-0.5,0.5,-0.5,0.5,results[i][0],results[i][1],1.0)
        intensity = spotlists[i].imax.mean() * ADU_correction
        intensities.append(intensity/1000.0)
    except:
        continue
slope, intercept, r_value, p_value, std_err = stats.linregress(exp_current[0:30],intensities[0:30])

xplot=linspace(0.0,7.0,100)
yplot = slope * xplot + intercept
scatter(exp_current,intensities)
plot(xplot, yplot, color='red')
text(0.5,70,"Linear Fit R^2 = %.5f"%r_value)
xlabel("Exp Time * Monitor Diode (nA-sec)")
ylabel("Modeled Peak Intensity (kADU)")
#xlim(0.0, 7.0)
#ylim(0.0,350.0)
savefig("Intensity_Check_Powell.png")

In [ ]:
x=array([1,2,3,4,5])
print x
resize(x,(2))
print x